In [37]:
from pathlib import Path # type: ignore
from freyja_plot import FreyjaPlotter
import plotly.io as pio
import numpy as np
import sys
if (module_path:=str(Path(".").absolute().resolve().parent)) not in sys.path:
    sys.path.insert(0, module_path)
from sample_info import colormap, summary_dict, expected, artic_runs as runs, plotting_dir, renameSamples

In [14]:
outdir = plotting_dir / "detection_plots/detection_plots"
outdir.mkdir(exist_ok=True)

In [15]:
file_map = {expected:"Expected"}
file_map.update({runs[tool]["WB"]:tool for tool in runs.keys()})
# for tool, files in runs.items():
#     for name,file in files.items():
#         if name == "WB":
#             # file_map[file] = f"{tool}: {name}"
#             file_map[file] = tool

plotter = FreyjaPlotter(file_map=file_map,summary_dict=summary_dict,colormap=colormap)
plotter = renameSamples(plotter)

### Dectection of expected lineages (Fig 5a in the paper)

In [16]:
fig1 = plotter.plotLineageDetections(show_counts=True,summarized=True,height=800,include_unexpected_lineages=True)
fn1 = outdir / "detection-summary.jpg"

In [17]:
pio.write_image(fig1, fn1, width=1400, height=1400, scale=2)
fig1.show()

### Detected lineage composition (Freyja) (Fig 5b in the paper)

In [18]:
file_map2 = {runs["Freyja"]["WB"]: "Freyja", expected:"Expected"}
freyja_plotter = FreyjaPlotter(file_map=file_map2,summary_dict=summary_dict,colormap=colormap)
freyja_plotter = renameSamples(freyja_plotter)
fig2 = freyja_plotter.plotLineageDetections(
    show_counts=False,
    summarized=True,
    height=600,
    include_unexpected_lineages=True,
    minimum_abundance=0.00,
    title="Freyja: Detected lineages (positive: correctly detected, negative: detected but unexpected)"
)
fn2 = outdir / "detection-w-false-pos-freyja-only.jpg"

In [19]:
pio.write_image(fig2, fn2, width=1400, height=700, scale=2)
fig2.show()

### Detection plots for each tool (used in supplementary materials)

In [86]:
fig3 = plotter.plotLineageDetections(summarized=True,show_counts=False,include_unexpected_lineages=True,minimum_abundance=0.01,height=6000)
# rename long names ('XBB.1.5* [Omicron (XBB.1.5.X)]' --> 'XBB.1.5*')
newnames = {}
for trace in fig3.data:
    element = trace.legendgroup
    if "*" in element:
        renamed = element.split(" ")[0].replace("*",".X")
        newnames[element] = renamed
        trace.text = np.array([renamed] * len(trace.text))
    else:
        newnames[element] = element
fig3.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fn3 = outdir / "detection-with-false-pos-0.01.jpg"

In [87]:
pio.write_image(fig3, fn3, width=1700, height=7000, scale=2)
fig3.show()